In [1]:
import tensorflow_datasets as tfds

In [2]:
import tensorflow as tf

In [3]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

In [4]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [5]:
encoder = info.features['text'].encoder

In [6]:
BUFFER_SIZE=10000
BATCH_SIZE=64

In [7]:
padded_shapes=([None], ())

In [8]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

In [9]:
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

In [10]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')])

In [11]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

In [12]:
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset, validation_steps=30)

Epoch 1/5
391/391 [==============================] - 268s 677ms/step - loss: 0.6460 - accuracy: 0.6106 - val_loss: 0.4677 - val_accuracy: 0.7911
Epoch 2/5
391/391 [==============================] - 271s 694ms/step - loss: 0.3520 - accuracy: 0.8582 - val_loss: 0.3658 - val_accuracy: 0.8562
Epoch 3/5
391/391 [==============================] - 275s 703ms/step - loss: 0.2540 - accuracy: 0.9042 - val_loss: 0.3580 - val_accuracy: 0.8630
Epoch 4/5
391/391 [==============================] - 285s 728ms/step - loss: 0.2170 - accuracy: 0.9214 - val_loss: 0.3290 - val_accuracy: 0.8724
Epoch 5/5
391/391 [==============================] - 280s 718ms/step - loss: 0.1872 - accuracy: 0.9354 - val_loss: 0.3478 - val_accuracy: 0.8677


In [13]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

In [14]:
def sample_predict(sentence, pad):
    encoded_sample_pred_text = encoder.encode(sentence)
    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
    return predictions

In [15]:
sample_text = ("This movie was awesome. The acting was incredible. Highly recommend!")

In [16]:
predictions = sample_predict(sample_text, pad=True) * 100

In [18]:
print("probability this is a positive review: %.2f" % predictions)

probability this is a positive review: 79.93
